# **structured Output using prompt template**

In [6]:
# Import required modules for using HuggingFace LLMs, environment variables, and prompt templates.

from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()


True

In [7]:
# Initialize the HuggingFaceEndpoint with the Mixtral model for text generation.

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation"
)

# Wrap the endpoint in a ChatHuggingFace object for chat-style interaction.
model = ChatHuggingFace(llm=llm)


In [8]:
# Define the first prompt template for generating a detailed report on a given topic.
# 1st prompt
template1 = PromptTemplate(
    template='write a detailed report on the topic "{topic}"',
    input_variables=["topic"]
)

# 2nd prompt

# Define the second prompt template for summarizing a given text in 5 lines.
template2 = PromptTemplate(
    template='write a 5 line summary on the following text. /n {text}',
    input_variables=["text"]

)


In [9]:
# Use the first prompt template to create a prompt about "Black Hole".

prompt1 = template1.invoke({"topic": "Black Hole"})


# Generate a detailed report on "Black Hole" using the model.
result = model.invoke(prompt1)

# Use the second prompt template to create a summary prompt from the generated report.
prompt2 = template2.invoke({"text": result.content})


# Generate a 5-line summary of the report using the model.
result = model.invoke(prompt2)

# Print the summary content.
print(result.content)

Here is a 5-line summary of the report on black holes:

Black holes are regions of spacetime where gravity is so strong that nothing, including light, can escape once it falls within a certain radius. They are formed when a massive star collapses in on itself, causing a massive amount of matter to be compressed into an incredibly small point. Black holes have several key properties, including mass, spin, charge, event horizon, and singularity. They have a profound impact on the surrounding environment, affecting the motion of nearby stars and the flow of matter and energy. Despite their elusive nature, black holes can be detected and studied through various methods, including X-rays, radio waves, gravitational waves, and astrometry.


# **Sturctured Output Using StrOutputparser**

In [ ]:
# Create a string output parser to extract plain text from the model's response.
parser = StrOutputParser()

# Build a chain that sequentially applies:
# 1. template1 (detailed report prompt)
# 2. model (LLM)
# 3. parser (extracts string)
# 4. template2 (summary prompt)
# 5. model (LLM)
# 6. parser (extracts summary string)


chain = template1 | model | parser | template2 | model | parser

In [ ]:
# Invoke the chain with the topic "artificial intelligence" to generate a detailed report,
# then summarize it in 5 lines, and print the final summary.

result = chain.invoke({"topic": "artificial intelligence"})

print(result)

Here is a 5-line summary of the text:

Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. AI has a rich history, dating back to the 1950s, and has evolved over the years with advances in computing power, data storage, and machine learning algorithms. Today, AI has a wide range of applications across various industries, including healthcare, finance, transportation, and education. However, AI also poses several challenges, including job displacement, bias and discrimination, privacy and security risks, and explainability and transparency issues. Despite these challenges, AI is expected to have a significant impact on various industries and aspects of our lives in the future.


# **Json Output Parser**

In [15]:
# Import required modules for using HuggingFace LLMs, environment variables, and prompt templates.

from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

load_dotenv()

True

In [16]:
# Initialize the HuggingFaceEndpoint with the Mixtral model for text generation.

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation"
)

# Wrap the endpoint in a ChatHuggingFace object for chat-style interaction.
model = ChatHuggingFace(llm=llm)

parser = JsonOutputParser()

In [ ]:
# Create a prompt template that asks for a fictional person's name, age, and city,
# and includes format instructions from the JSON output parser to ensure structured output.

template = PromptTemplate(
    template='Give me the name, age and city of a fictional person.\n {format_instruction} ',
    input_variables=[],
    partial_variables={"format_instruction": parser.get_format_instructions()}
    
)



In [ ]:
# Format the prompt using the template, send it to the model, and parse the response as JSON.
prompt = template.format()

result = model.invoke(prompt)

final_result = parser.parse(result.content)

print(final_result)

print(type(final_result))

{'name': 'Emily Wilson', 'age': 32, 'city': 'Seattle'}
<class 'dict'>


#  **Json Output Parser using chain**

In [ ]:
# Build a chain that combines the prompt template, model, and JSON output parser.

chain = template | model | parser

result = chain.invoke({})

print(result)

Here is a JSON object with the requested information:

```
{
  "name": "Emily Wilson",
  "age": 32,
  "city": "Seattle"
}
```


# **Unstructured output from LLMS**

In [ ]:
# Create a prompt template that asks for five facts about a given topic,
# using format instructions from the JSON output parser for structured output.

template = PromptTemplate(
    
    template='Give me five facts about {topic} \n {format_instruction}',
    
    input_variables=['topic'],
   
    partial_variables={"format_instruction": parser.get_format_instructions()}
    
)

In [ ]:
# Build a chain with the template, model, and parser to get five facts about "machine learning".

chain = template | model | parser

result = chain.invoke({'topic': 'machine learning'})

print(result)

{'fact1': {'title': 'Machine Learning is a Subset of Artificial Intelligence', 'description': 'Machine learning is a subset of artificial intelligence that involves training algorithms to learn from data and make predictions or decisions without being explicitly programmed.'}, 'fact2': {'title': 'Machine Learning is Used in Many Applications', 'description': 'Machine learning is used in a wide range of applications, including image and speech recognition, natural language processing, recommender systems, and predictive maintenance.'}, 'fact3': {'title': 'Machine Learning Algorithms Can Be Supervised or Unsupervised', 'description': 'Machine learning algorithms can be supervised, meaning they are trained on labeled data, or unsupervised, meaning they are trained on unlabeled data and must find patterns or structure on their own.'}, 'fact4': {'title': 'Deep Learning is a Type of Machine Learning', 'description': 'Deep learning is a type of machine learning that involves the use of artifi

# **Structured Output Parser**

Structured output parser is an output parser in langchain that helps extract
structured Json data LLM based on predefined field schemas.

it works by defining a list of fields (ResponseSchema) that the model should return, ensuring the output follows a structured format.

In [19]:
# Import required modules for using HuggingFace LLMs, environment variables, and prompt templates.

from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

load_dotenv()

True

In [ ]:
# Initialize the HuggingFaceEndpoint with the meta-llama model for text generation.

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation"
)

# Wrap the endpoint in a ChatHuggingFace object for chat-style interaction.
model = ChatHuggingFace(llm=llm)

In [ ]:
# Define a schema for structured output, specifying five fields for five facts about a topic.

schema = [
    ResponseSchema(name="fact_1", description="Fact 1 about the topic"),
    ResponseSchema(name="fact_2", description="Fact 2 about the topic"),
    ResponseSchema(name="fact_3", description="Fact 3 about the topic"),
    ResponseSchema(name="fact_4", description="Fact 4 about the topic"),
    ResponseSchema(name="fact_5", description="Fact 5 about the topic"),
    
    
]

In [ ]:
# Create a structured output parser using the defined schema.

parser = StructuredOutputParser.from_response_schemas(schema)

In [ ]:
# Create a prompt template that asks for five facts about a topic,
# and includes format instructions for the structured output parser.

template = PromptTemplate(
    template='Give me five facts about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={"format_instruction": parser.get_format_instructions()}
)

In [ ]:
# Generate a prompt for the topic "deep learning", invoke the model,
# parse the structured response, and print the final result.

prompt = template.invoke({"topic": "deep learning"})

result = model.invoke(prompt)

final_result = parser.parse(result.content)

print(final_result)


{'fact_1': 'Deep learning is a subset of machine learning that uses artificial neural networks to analyze data and make predictions or decisions. It is inspired by the structure and function of the human brain.', 'fact_2': 'Deep learning models can learn to recognize patterns and relationships in data, such as images, speech, and text, by adjusting the connections between artificial neurons and the strength of these connections.', 'fact_3': 'Deep learning has achieved state-of-the-art results in many areas, including image recognition, speech recognition, natural language processing, and game playing, outperforming traditional machine learning methods and human experts in some cases.', 'fact_4': 'Deep learning models are typically trained using large amounts of data and computational resources, and can be prone to overfitting, where the model becomes too specialized to the training data and fails to generalize well to new data.', 'fact_5': "Deep learning has many applications in indust

In [ ]:
# Build a chain to get five structured facts about "quantum computing" and print the result.

chain = template | model | parser

result = chain.invoke({'topic': 'quantum computing'})

print(result)

{'fact_1': 'Quantum computers use quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. This allows them to solve certain problems much faster than classical computers.', 'fact_2': 'Quantum computers can exist in multiple states simultaneously, which is known as a qubit (quantum bit). This is in contrast to classical computers, which can only exist in one of two states (0 or 1).', 'fact_3': 'Quantum computers can perform certain calculations, such as factoring large numbers, much faster than classical computers. This has significant implications for cryptography and encryption.', 'fact_4': "The first quantum computer was built in 1982 by physicist David Deutsch. However, it wasn't until the 1990s that the concept of quantum computing began to gain widespread attention.", 'fact_5': 'Quantum computers are still in the early stages of development, and many technical challenges need to be overcome before they can be widely used. However, compa

# **Pydantic Output Parser**

In [ ]:
# Import required modules for using HuggingFace LLMs, environment variables, and prompt templates

from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
load_dotenv()

True

In [ ]:
# Initialize the HuggingFaceEndpoint with the meta-llama model for text generation.
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

In [28]:
class Person(BaseModel):

    name: str = Field(description='Name of the person')
    age: int = Field(gt=18, description='Age of the person')
    city: str = Field(description='Name of the city the person belongs to')

In [29]:
parser = PydanticOutputParser(pydantic_object=Person)

In [30]:
template = PromptTemplate(
    template='Generate the name, age and city of a fictional {place} person \n {format_instruction}',
    input_variables=['place'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

In [31]:
chain = template | model | parser

final_result = chain.invoke({'place':'Pakistan'})

print(final_result)

name='Ali Khan' age=32 city='Karachi'


In [ ]:
# Generate and print a prompt for a fictional person from the United States using the template.

prompt = template.invoke({"place": "united states"})

print(prompt)

text='Generate the name, age and city of a fictional united states person \n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "Name of the person", "title": "Name", "type": "string"}, "age": {"description": "Age of the person", "exclusiveMinimum": 18, "title": "Age", "type": "integer"}, "city": {"description": "Name of the city the person belongs to", "title": "City", "type": "string"}}, "required": ["name", "age", "city"]}\n```'


In [ ]:
# Invoke the model with the generated prompt, parse the response using the Pydantic parser,
# and print the final structured result.

result = model.invoke(prompt)

final_result = parser.parse(result.content)

print(final_result)

name='Emily Wilson' age=32 city='Denver'
